https://www.kaggle.com/code/jimbothehut/content-based-manga-recommender-system/notebook

https://github.com/kk7nc/Text_Classification#rocchio-classification

https://queryunderstanding.com/query-expansion-2d68d47cf9c8


In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer,  CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
import pandas as pd
import nltk
import math
import string
from itertools import groupby
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
def remove_stop_words(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    return text

def lowercase(text):
  return text.lower()

def preprocess(text):
  text = lowercase(text)
  text = remove_punctuation(text)
  text = remove_stop_words(text)
  return text

In [ ]:
keywords = open("keywords.txt").readlines()
keywords = [x.strip() for x in keywords]

In [ ]:
documents = open("documents.txt", encoding='utf-8-sig').read().split('\n\n')
documents = [x.replace('\n', ' ').replace('...', '').replace('�', '').replace('.  ', '.') for x in documents]

In [ ]:
process_docs = [preprocess(x).lower() for x in documents]

In [ ]:
def wordcount(doc):
  word_count = defaultdict(int)
  for sentence in doc:
    for word in sentence.split():
      word_count[word] += 1
  word_count = dict(sorted(word_count.items(), key = lambda x : x[1], reverse = True))
  return word_count

In [ ]:
def zipf_plot(doc):
  word_count = wordcount(doc)
  plt.figure(figsize = (50, 10))
  plt.tight_layout()
  plt.xlabel('Words', fontsize = 27)
  plt.ylabel('Frequency', fontsize = 27)
  plt.xticks(rotation=90, fontsize = 5)
  plt.bar(word_count.keys(), word_count.values())

In [ ]:
zipf_plot(process_docs)

In [ ]:
def heaps_plot(doc):
    frequency = []
    for i in doc:
        f = len(i)
        l = []
        c = 0
        for j in i:
          if j not in l:
            c+=1
            l.append(j)
        frequency.append([f, c])
    frequency.sort(reverse = True)
    plt.plot(frequency[:][0], frequency[:][1])

In [ ]:
vectorizer = TfidfVectorizer(vocabulary=keywords)
X = vectorizer.fit_transform(process_docs)
print(X.shape)

(88, 120)


In [ ]:
def cosine_sim(query, X):
    q_tfidf = vectorizer.fit_transform([query])
    results = cosine_similarity(X,q_tfidf).reshape((-1,))
    return results

In [ ]:
def query_search(results):
    print("Top 10 Documents matched: ")
    j = 0
    res = []
    for i in results.argsort()[-10:][::-1]:
        j += 1
        print("Document {:02d}: ".format(j), documents[i])
        res.append(documents[i])
    return res

In [ ]:
def get_relevance(results):
    res_bool = []
    for x in range(len(results)):
        ch = input(f"Type R or N for document {x+1} being relevant and non-relevant respectively: ")
        if ch == 'R' or ch == 'r':
            res_bool.append(True)
        else:
            res_bool.append(False)
    return res_bool

In [ ]:
def get_vectors(docs):
    vec = []
    for x in docs:
        i = documents.index(x)
        vec.append(X[i])
    return vec

In [ ]:
def rocchio_rel(query, vectors, rel_vec, alpha=1, beta=0.8, gamma=0.1):
    rels_ = [vectors[i] for i in range(len(vectors)) if rel_vec[i] == True]
    non_rels = [vectors[i] for i in range(len(vectors)) if rel_vec[i] == False]
    C1 = sum(rels_) / len(rels_)
    C2 = sum(non_rels) / len(non_rels)
    q = vectorizer.fit_transform([query])
    q1 = alpha * q + beta * C1 - gamma * C2
    results = cosine_similarity(X,q1).reshape((-1,))
    return results

In [ ]:
query = 'neural machine learning'
query_docs = query_search(cosine_sim(query, X))

Top 10 Documents matched: 
Document 01:  15-681 and 15-781 Machine Learning Machine Learning, 15:681 and 15:781, Fall 1998.This course covers the theory and practice of machine learning from a variety of perspectives.
Document 02:  C4.5: Programs for Machine Learning By Author. C4.5: Programs for Machine Learning J. Ross Quinlan, The Morgan Kaufmann Series in Machine  
Document 03:  Machine Learning Machine Learning, THE  Machine learning refers to a system capable of the autonomous acquisition and integration of knowledge. This  
Document 04:  Machine Learning 6.858/18.428: Machine Learning. Available Lecture Notes.Defining models for machine learning. Learning conjunctions in the mistake-bounded model.
Document 05:  Machine Learning 6.858/18.428: Machine Learning.This course deals with the following topics: Formal models of machine learning; Learning concepts from examples;  
Document 06:  Machine Learning FAQ Machine Learning FAQ. Machine Learning Frequently Asked Questions. Click H

In [ ]:
relevance = get_relevance(query_docs)
tfidf_vectors = get_vectors(query_docs)
rocchio_results = rocchio_rel(query, tfidf_vectors, relevance)
q_docs = query_search(rocchio_results)

Type R or N for document 1 being relevant and non-relevant respectively: R
Type R or N for document 2 being relevant and non-relevant respectively: N
Type R or N for document 3 being relevant and non-relevant respectively: R
Type R or N for document 4 being relevant and non-relevant respectively: R
Type R or N for document 5 being relevant and non-relevant respectively: R
Type R or N for document 6 being relevant and non-relevant respectively: N
Type R or N for document 7 being relevant and non-relevant respectively: N
Type R or N for document 8 being relevant and non-relevant respectively: R
Type R or N for document 9 being relevant and non-relevant respectively: R
Type R or N for document 10 being relevant and non-relevant respectively: R
Top 10 Documents matched: 
Document 01:  15-681 and 15-781 Machine Learning Machine Learning, 15:681 and 15:781, Fall 1998.This course covers the theory and practice of machine learning from a variety of perspectives.
Document 02:  C4.5: Programs fo

In [ ]:
countvect = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
count_docs = countvect.fit_transform(process_docs)

In [ ]:
NUM_TOPICS = 10
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online',verbose=True)
data_lda = lda.fit_transform(count_docs)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [ ]:
def selected_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names_out()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])

In [ ]:
print('LDA Model: ')
selected_topics(lda, countvect)

LDA Model: 
Topic 0:
[('group', 8.985601734978186), ('research', 6.522558818474625), ('description', 6.250855557102563), ('computer', 5.042182784458683), ('algorithms', 4.026352816259424), ('classification', 2.616914148683244), ('home', 2.1959980032577393), ('science', 1.9489734393136977), ('page', 1.4417100403100485), ('information', 1.2995399185728993)]
Topic 1:
[('research', 17.83141639820532), ('group', 11.88433318210909), ('systems', 5.041024491991795), ('description', 4.84930073346815), ('university', 3.46782086757878), ('new', 2.5905503175603095), ('information', 2.057256239727811), ('overview', 1.8964796849256063), ('pages', 1.8382749340652973), ('computer', 1.7560203016429135)]
Topic 2:
[('page', 12.230608657719868), ('home', 10.274729230834941), ('pages', 4.311083738882294), ('description', 4.286673016904455), ('artificial', 3.517065184524415), ('intelligence', 3.511844774029209), ('new', 1.9013320747932734), ('people', 1.535343806661428), ('web', 1.2416971291645875), ('appli